In [1]:
import random, numpy, math, gym

from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

import gym_trader.envs.trader as make

Using TensorFlow backend.


### Brain

In [83]:
class Brain:
    
    def __init__(self, stateCnt, actionCnt):
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt
        
        self.model = self._createModel()

    
    def _createModel(self):
        
        model = Sequential()

        model.add(Dense(12, activation='relu', input_dim=stateCnt))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(units=actionCnt, activation='linear'))

        opt = RMSprop(lr=0.01)
        model.compile(loss='mse', optimizer=opt)
        
        return model
    
    def train(self, x, y, epoch=1, verbose=0):
        self.model.fit(x, y, batch_size = 64, epochs=epoch, verbose=verbose)
    
    def predict(self, s):
        return self.model.predict(s)
    
    def predictOne(self, s):
        return self.predict(s.reshape(1, self.stateCnt)).flatten()

### Memory

In [84]:
class Memory:
    samples = []
    
    def __init__(self, capacity):
        self.capacity = capacity
    
    def add(self, sample):
        self.samples.append(sample)
        
        if len(self.samples) > self.capacity:
            self.samples.pop(0)
    
    def sample(self, n):
        n = min(n, len(self.samples))
        return random.sample(self.samples, n)

### Agent

In [85]:
MEMORY_CAPACITY = 100000
BATCH_SIZE = 32

GAMMA = 0.99

MAX_EPSILON = 1
MIN_EPSILON = 0.01
LAMBDA = 0.001 

In [86]:
class Agent:
    steps = 0
    epsilon = MAX_EPSILON
    
    def __init__(self, stateCnt, actionCnt):
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt
        
        self.brain = Brain(stateCnt, actionCnt)
        self.memory = Memory(MEMORY_CAPACITY)
        
    def act(self, s):
        if random.random() < self.epsilon:
            return random.randint(0, self.actionCnt - 1)
        else:
            return numpy.argmax(self.brain.predictOne(s))
    
    def observe(self, sample):
        self.memory.add(sample)
        
        self.steps += 1
        self.epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * math.exp(-LAMBDA * self.steps)
        
        
    def replay(self):
        batch = self.memory.sample(BATCH_SIZE)
        batchLen = len(batch)
        
        no_state = numpy.zeros(self.stateCnt)
        
        states = numpy.array( [o[0] for o in batch] )
        next_state = numpy.array( [(no_state if o[3] is None else o[3])for o in batch] )
    
        p = self.brain.predict(states)
        next_p = self.brain.predict(next_state)
        
        x = numpy.zeros((batchLen, self.stateCnt))
        y = numpy.zeros((batchLen, self.actionCnt))
        
        for i in range(batchLen):
            o = batch[i]
            state = o[0];
            action = o[1];
            reward = o[2];
            next_state = o[3];
            
            t = p[i]
            if next_state is None:
                t[action] = reward
            else:
                t[action] = reward + GAMMA * numpy.amax(next_p[i])
            
            x[i] = state
            y[i] = t
            
        self.brain.train(x, y)

### Environment

In [87]:
class Environment:
    
    def __init__(self):
        self.env = make.TraderEnv()
        
    def run(self, agent):
        state = self.env.reset()
        R = 0
        
        while True:
             
            action = agent.act(state)
            
            s_, reward, done, info = self.env.step(action)
            
            if done:
                s_ = None
                
            agent.observe( (state, action, reward, s_) )
            agent.replay()
            
            state = s_
            
            R += reward
            
            if done:
                break
                
        print("Total reward: ", R)


### Main

In [ ]:
env = Environment()

stateCnt = env.env.observation_space.n
actionCnt = env.env.action_space.n

agent = Agent(stateCnt, actionCnt)

try:
    while True:
        env.run(agent)
finally:
    agent.brain.model.save("model.h5")        

      


Total reward:  -169
Total reward:  -11491
Total reward:  -786
Total reward:  -97
Total reward:  -10
Total reward:  -202
Total reward:  -58
Total reward:  -189
Total reward:  -178
Total reward:  -116
Total reward:  -63
Total reward:  -81
Total reward:  -128
Total reward:  -178
Total reward:  -205
Total reward:  -86
Total reward:  -76
Total reward:  -134
Total reward:  -45
Total reward:  -92
Total reward:  -70
Total reward:  -59
Total reward:  -71
Total reward:  -103
Total reward:  -90
Total reward:  7
Total reward:  -66
Total reward:  -85
Total reward:  -101
Total reward:  -80
Total reward:  -87
Total reward:  -53
Total reward:  -71
Total reward:  -66
Total reward:  -61
Total reward:  -31
Total reward:  -83
Total reward:  -70
Total reward:  -47
Total reward:  -64
Total reward:  -81
Total reward:  -151
Total reward:  -42
Total reward:  -55
Total reward:  -24
Total reward:  -73
Total reward:  -38
Total reward:  -47
Total reward:  -39
Total reward:  -25
Total reward:  -66
Total reward:  -7

Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -43
Total reward:  -44
Total reward:  -41
Total reward:  -41
Total reward:  -45
Total reward:  -41
Total reward:  -43
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -53
Total reward:  -79
Total reward:  -66
Total reward:  -41
Total reward:  -30
Total reward:  -69
Total reward:  -66
Total reward:  -76
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -24
Total reward:  -66
Total reward:  -138
Total reward:  -41
Total reward:  -69
Total reward:  -53
Total reward:  -40
Total reward:  -41
Total reward:  -40
Total reward:  -44
Total reward:  -46
Total reward:  -142
Total reward:  -63
Total reward:  -40
Total reward:  -49
Total rewa

Total reward:  -272
Total reward:  -41
Total reward:  -85
Total reward:  -60
Total reward:  -75
Total reward:  -650
Total reward:  -294
Total reward:  -41
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -40
Total reward:  -45
Total reward:  -41
Total reward:  -37
Total reward:  -45
Total reward:  -33
Total reward:  -35
Total reward:  -128
Total reward:  -90
Total reward:  -66
Total reward:  -66
Total reward:  -110
Total reward:  -66
Total reward:  -194
Total reward:  -70
Total reward:  -97
Total reward:  -75
Total reward:  -271
Total reward:  -75
Total reward:  -24
Total reward:  -11
Total reward:  -165
Total reward:  -41
Total reward:  -41
Total reward:  -284
Total reward:  -86
Total reward:  -76
Total reward:  -80
Total reward:  -328
Total reward:  -66
Total reward:  -230
Total reward:  -871
Total reward:  -66
Total reward:  -63
Total reward:  -138
Total reward:  -164
Total reward:  -41
Total reward:  -97
Total reward:  -72
Total reward:  -6

Total reward:  -66
Total reward:  -66
Total reward:  -67
Total reward:  -45
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -375
Total reward:  -66
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -211
Total reward:  -66
Total reward:  -778
Total reward:  -66
Total reward:  -313
Total reward:  -92
Total reward:  -66
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total re

Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -65
Total reward:  -41
Total reward:  -65
Total reward:  -66
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -47
Total reward:  -41
Total reward:  -41
Total reward:  -73
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward

...

Total reward:  -43
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -89
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -49
Total reward:  -41
Total reward:  -39
Total reward:  -281
Total reward:  -41
Total reward:  -41
Total reward:  -91
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -65
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -40
Total reward:  -39
Total reward:  -41
Total reward:  -41
Total reward:  -55
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -44
Total rewar

Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -59
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -20
Total reward:  -39
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -45
Total reward:  -41
Total reward:  -41
Total reward:  -43
Total reward:  -14
Total reward:  -47
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward:  -65
Total reward:  -36
Total reward:  -36
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -44
Total reward:  -41
Total reward:  -52
Total reward:  -41
Total reward:  -66
Total reward:  -49
Total reward:  -46
Total reward:  -39
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward

Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -75
Total reward:  -41
Total reward:  -41
Total reward:  -52
Total reward:  -36
Total reward:  -12
Total reward:  -41
Total reward:  -64
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward:  -47
Total reward:  -45
Total reward:  -66
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -81
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -45
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -65
Total reward:  -35
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -67
Total reward:  -64
Total reward:  -66
Total reward:  -41
Total reward:  -65
Total reward:  -55
Total reward:  -41
Total reward:  -41
Total reward

Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -73
Total reward:  -65
Total reward:  -64
Total reward:  -41
Total reward:  -71
Total reward:  -43
Total reward:  -41
Total reward:  -85
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -46
Total reward:  -41
Total reward:  -33
Total reward:  -43
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward

Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -65
Total reward:  -66
Total reward:  -116
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -65
Total reward:  -52
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward:  -65
Total reward:  -41
Total reward:  -41
Total reward:  -24
Total reward:  -41
Total reward:  -47
Total reward:  -66
Total reward:  -65
Total reward:  -66
Total reward:  -41
Total reward:  -41
Total reward:  -64
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total reward:  -57
Total reward:  -38
Total reward:  -41
Total reward:  -41
Total reward:  -66
Total reward:  -66
Total reward:  -66
Total reward:  -41
Total reward:  -52
Total reward:  -12
Total reward:  -41
Total reward:  -41
Total reward:  -41
Total rewar